### Imports

In [ ]:
import pandas as pd

### Load data files


In [ ]:
classes_trainable = pd.read_csv(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\docs\classes-trainable.csv')
class_descriptions = pd.read_csv(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\docs\class-descriptions.csv')
train_human_labels = pd.read_csv(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\docs\train_human_labels.csv')
train_machine_labels = pd.read_csv(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\docs\train_machine_labels.csv')
train_bounding_boxes = pd.read_csv(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\docs\train_bounding_boxes.csv')

### Analyze

In [ ]:
print('len(classes_trainable) = {}\n'.format(len(classes_trainable)))
print(classes_trainable.head())

In [10]:
print('len(class description) = {}\n'.format(len(class_descriptions)))
print(class_descriptions.head())

len(class description) = 19987

    label_code          description
0  /m/0100nhbf     Sprenger's tulip
1  /m/0104x9kv             Vinegret
2  /m/0105jzwx            Dabu-dabu
3  /m/0105ld7g  Pistachio ice cream
4  /m/0105lxy5                 Woku


In [14]:
print('len(train_human_labels) = {}'.format(len(train_human_labels)))
print('len(train_machine_labels) = {}'.format(len(train_machine_labels)))
print('\n')
print(train_human_labels.head())
print('\n')
print(train_machine_labels.head())

len(train_human_labels) = 8036466
len(train_machine_labels) = 15259186


            ImageID                    Source  LabelName  Confidence
0  000002b66c9c498e  crowdsource-verification  /m/01kcnl           1
1  000002b66c9c498e              verification   /m/012mj           1
2  000002b66c9c498e              verification  /m/012yh1           1
3  000002b66c9c498e              verification  /m/014sv8           1
4  000002b66c9c498e              verification  /m/016c68           1


            ImageID   Source   LabelName  Confidence
0  000002b66c9c498e  machine    /m/05_4_         0.7
1  000002b66c9c498e  machine    /m/0krfg         0.7
2  000002b66c9c498e  machine   /m/01kcnl         0.5
3  000002b97e5471a0  machine  /m/05_5t0l         0.9
4  000002b97e5471a0  machine    /m/0cgh4         0.8


### Top labels

In [30]:
train_human_labels_values_count = train_human_labels['LabelName'].value_counts()
print(train_human_labels_values_count.head())

/m/01g317    807090
/m/09j2d     610840
/m/04yx4     378360
/m/0dzct     331942
/m/07j7r     315026
Name: LabelName, dtype: int64


### Create files with only trainable labels

In [51]:
train_human_labels_trainable = train_human_labels[train_human_labels['LabelName'].isin(classes_trainable['label_code'])]
train_machine_labels_trainable = train_machine_labels[train_machine_labels['LabelName'].isin(classes_trainable['label_code'])]

,ImageID,Source,LabelName,Confidence
0,000002b66c9c498e,crowdsource-verification,/m/01kcnl,1
1,000002b66c9c498e,verification,/m/012mj,1
2,000002b66c9c498e,verification,/m/012yh1,1
3,000002b66c9c498e,verification,/m/014sv8,1
4,000002b66c9c498e,verification,/m/016c68,1


In [20]:
print('len(train_human_labels_trainable) = {}'.format(len(train_human_labels_trainable)))
print('len(train_machine_labels_trainable) = {}'.format(len(train_machine_labels_trainable)))

len(train_human_labels_trainable) = 7117742
len(train_machine_labels_trainable) = 13409172


In [52]:
train_human_labels_trainable.to_csv(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\docs\train_human_labels_trainable.csv', index=False)
train_machine_labels_trainable.to_csv(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\docs\train_machine_labels_trainable.csv', index=False)

### Create files with top trainable lables (10/100)

In [32]:
top_10_labels = train_human_labels_values_count.head(10)
top_100_lables = train_human_labels_values_count.head(100)

In [48]:
train_human_labels_trainable_top_10 = train_human_labels_trainable[train_human_labels_trainable['LabelName'].isin(list(top_10_labels.index))]
train_human_labels_trainable_top_100 = train_human_labels_trainable[train_human_labels_trainable['LabelName'].isin(list(top_100_labels.index))]

In [54]:
train_human_labels_trainable_top_10.to_csv(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\docs\train_human_labels_trainable_top_10.csv', index=False)
train_human_labels_trainable_top_100.to_csv(r'\\ger\ec\proj\ha\RSG\FacePublicDatasets\OpenImages\InclusiveChallenge\docs\train_human_labels_trainable_top_100.csv', index=False)